In [2]:
import sys
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
import csv
import math
import operator
import data_processing_training

In [3]:
coupon_Id = "51800000050"
df_coupon = pd.read_csv('coupon.csv', dtype={'COUPON_UPC': str, 'CAMPAIGN': str, 'PRODUCT_ID': str})
campaigns = data_processing_training.get_campaigns_for_coupon(coupon_Id, df_coupon)
product_list = data_processing_training.get_products_for_coupon(coupon_Id, df_coupon)

In [4]:
del df_coupon

In [5]:
df_campaign_table = pd.read_csv('campaign_table.csv', dtype={'household_key': str, 'CAMPAIGN': str})
df_campaign_desc = pd.read_csv('campaign_desc.csv', dtype={'CAMPAIGN': str})

In [6]:
hh_start_dates = data_processing_training.get_households_for_campaigns(campaigns, df_campaign_table, df_campaign_desc)
len(hh_start_dates)

629

In [7]:
del df_campaign_table

In [8]:
hh_start_dates.head()

,DESCRIPTION_x,household_key,CAMPAIGN,DESCRIPTION_y,START_DAY,END_DAY
349,TypeB,1,12,TypeB,477,509
121,TypeA,1003,26,TypeA,224,264
120,TypeA,1012,26,TypeA,224,264
453,TypeB,1014,12,TypeB,477,509
119,TypeA,1018,26,TypeA,224,264


In [9]:
hh_start_dates.drop(columns=['DESCRIPTION_x', 'DESCRIPTION_y'], inplace=True)
hh_start_dates.head()

,household_key,CAMPAIGN,START_DAY,END_DAY
349,1,12,477,509
121,1003,26,224,264
120,1012,26,224,264
453,1014,12,477,509
119,1018,26,224,264


In [10]:
df_transactions = pd.read_csv('transaction_data.csv', dtype={'BASKET_ID': str, 'PRODUCT_ID': str, 'household_key': str, 'DAY': str})
df_causal = pd.read_csv('causal_data.csv', dtype={'PRODUCT_ID': str, 'STORE_ID': int, 'WEEK_NO': float, 'display': str})

In [11]:
len(df_transactions['BASKET_ID'].unique())

276484

In [12]:
len(df_transactions['household_key'].unique())

2500

In [13]:
hh_list = hh_start_dates['household_key'].unique()
transactions_hh = df_transactions[df_transactions['household_key'].isin(hh_list)]
len(transactions_hh)

1189675

In [14]:
len(df_transactions)

2595732

In [15]:
trans_merge = df_transactions.merge(hh_start_dates, on='household_key', how='left')
len(trans_merge)

2595732

In [16]:
len(trans_merge[trans_merge['START_DAY'].isnull()])

1406057

In [17]:
trans_merge.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,CAMPAIGN,START_DAY,END_DAY
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,NaN,NaN,NaN
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,NaN,NaN,NaN
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,NaN,NaN,NaN
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,NaN,NaN,NaN
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,NaN,NaN,NaN


In [18]:
trans_merge['START_DAY'].fillna(10000, inplace=True)
#trans_merge = trans_merge[trans_merge['DAY'].astype(float) < trans_merge['START_DAY']]
len(trans_merge[trans_merge['START_DAY'].isnull()])

0

In [19]:
trans_merge.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,CAMPAIGN,START_DAY,END_DAY
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,NaN,10000.0,NaN
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,NaN,10000.0,NaN
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,NaN,10000.0,NaN
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,NaN,10000.0,NaN
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,NaN,10000.0,NaN


In [20]:
len(trans_merge[trans_merge['START_DAY'] == 10000])

1406057

In [21]:
len(trans_merge[trans_merge['household_key'] == "2375"])

745

In [22]:
transactions_hh = df_transactions[~df_transactions['household_key'].isin(hh_list)]
transactions_hh.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [23]:
transactions_hh = df_transactions[df_transactions['household_key'].isin(hh_list)]
transactions_hh.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
11,1364,26984896261,1,842930,1,2.19,31742,0.00,1520,1,0.0,0.0
12,1364,26984896261,1,897044,1,2.99,31742,-0.40,1520,1,0.0,0.0
13,1364,26984896261,1,920955,1,3.09,31742,0.00,1520,1,0.0,0.0
14,1364,26984896261,1,937406,1,2.50,31742,-0.99,1520,1,0.0,0.0
15,1364,26984896261,1,981760,1,0.60,31742,-0.79,1520,1,0.0,0.0


In [24]:
trans_merge_2375 = trans_merge[trans_merge['household_key'] == "2375"]
trans_merge_2375.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,CAMPAIGN,START_DAY,END_DAY
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,NaN,10000.0,NaN
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,NaN,10000.0,NaN
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,NaN,10000.0,NaN
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,NaN,10000.0,NaN
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,NaN,10000.0,NaN


In [25]:
len(trans_merge[trans_merge['household_key'] == "1364"])

912

In [26]:
trans_merge_higher = trans_merge[trans_merge['DAY'].astype(float) < trans_merge['START_DAY']]
len(trans_merge_higher)

2068915

In [27]:
trans_merge_lower = trans_merge[trans_merge['DAY'].astype(float) >= trans_merge['START_DAY']]
len(trans_merge_lower)

526817

In [28]:
len(trans_merge)

2595732

In [29]:
len(trans_merge_higher[trans_merge_higher['household_key'] == "1364"])

305

In [30]:
len(trans_merge_lower[trans_merge_lower['household_key'] == "1364"])

607

In [31]:
len(trans_merge_higher[trans_merge_higher['household_key'] == "2375"])

745

In [32]:
len(trans_merge_lower[trans_merge_lower['household_key'] == "2375"])

0

In [33]:
trans_merge_higher.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,CAMPAIGN,START_DAY,END_DAY
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,NaN,10000.0,NaN
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,NaN,10000.0,NaN
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,NaN,10000.0,NaN
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,NaN,10000.0,NaN
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,NaN,10000.0,NaN


In [34]:
len(trans_merge_higher)

2068915

In [35]:
len(df_causal)

36786524

In [36]:
df_causal.head()

,PRODUCT_ID,STORE_ID,WEEK_NO,display,mailer
0,26190,286,70.0,0,A
1,26190,288,70.0,0,A
2,26190,289,70.0,0,A
3,26190,292,70.0,0,A
4,26190,293,70.0,0,A


In [37]:
trans_merge_higher = trans_merge_higher.merge(df_causal, on=['STORE_ID', 'PRODUCT_ID'], how="left")

In [38]:
len(trans_merge_higher)

25719778

In [39]:
trans_merge_higher.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO_x,COUPON_DISC,COUPON_MATCH_DISC,CAMPAIGN,START_DAY,END_DAY,WEEK_NO_y,display,mailer
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,NaN,10000.0,NaN,9.0,0,F
1,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,NaN,10000.0,NaN,14.0,0,F
2,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,NaN,10000.0,NaN,32.0,0,F
3,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,NaN,10000.0,NaN,35.0,0,F
4,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,NaN,10000.0,NaN,36.0,0,A


In [40]:
trans_merge_higher['unique'] = trans_merge_higher['PRODUCT_ID'].astype(str) + trans_merge_higher['STORE_ID'].astype(str)
trans_merge_higher.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO_x,COUPON_DISC,COUPON_MATCH_DISC,CAMPAIGN,START_DAY,END_DAY,WEEK_NO_y,display,mailer,unique
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,NaN,10000.0,NaN,9.0,0,F,1004906364
1,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,NaN,10000.0,NaN,14.0,0,F,1004906364
2,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,NaN,10000.0,NaN,32.0,0,F,1004906364
3,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,NaN,10000.0,NaN,35.0,0,F,1004906364
4,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,NaN,10000.0,NaN,36.0,0,A,1004906364


In [41]:
len(trans_merge_higher['unique'].unique())

814278

In [42]:
trans_merge_higher['CUSTOMER_PAID'] = trans_merge_higher['SALES_VALUE'] + trans_merge_higher['COUPON_DISC']
trans_merge_higher['WEEKS_TO_MAILER'] = trans_merge_higher['WEEK_NO_x'] - trans_merge_higher['WEEK_NO_y']
trans_merge_higher['WEEKS_TO_MAILER'].fillna(1000, inplace=True)

In [43]:
len(trans_merge_higher['unique'].unique())

814278

In [44]:
trans_merge_higher.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO_x,...,COUPON_MATCH_DISC,CAMPAIGN,START_DAY,END_DAY,WEEK_NO_y,display,mailer,unique,CUSTOMER_PAID,WEEKS_TO_MAILER
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,...,0.0,NaN,10000.0,NaN,9.0,0,F,1004906364,1.39,-8.0
1,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,...,0.0,NaN,10000.0,NaN,14.0,0,F,1004906364,1.39,-13.0
2,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,...,0.0,NaN,10000.0,NaN,32.0,0,F,1004906364,1.39,-31.0
3,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,...,0.0,NaN,10000.0,NaN,35.0,0,F,1004906364,1.39,-34.0
4,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,...,0.0,NaN,10000.0,NaN,36.0,0,A,1004906364,1.39,-35.0


In [45]:
len(trans_merge_higher)

25719778

In [ ]:
trans_merge_higher.sort_values(['household_key', 'BASKET_ID', 'WEEKS_TO_MAILER'], inplace=True)